In [ ]:
pip install sqlite3

### Crear base de datos con SQLite

Primero vamos a crear una base de datos

In [ ]:
import sqlite3

conn = sqlite3.connect('todo.db')
conn.execute("CREATE TABLE todo (id INTEGER PRIMARY KEY, task char(100) NOT NULL, status boll NOT NULL)")
conn.execute("INSERT INTO todo (task, status)VALUES('Crear hola mundo en bottlepy', 1)")
conn.execute("INSERT INTO todo (task, status)VALUES('Documentar su creacion en jupyter',1)")
conn.execute("INSERT INTO todo (task, status)VALUES('Crar un proyecto nuevo de una lista ToDO',1)")
conn.execute("INSERT INTO todo (task, status)VALUES('Documentar creacion de ToDo en jupyter',0)")
conn.commit()

In [ ]:
import sqlite3
from bottle import Bottle, run

app= Bottle()

@app.route('/todo')
@app.route('/')
def todo_list():
    connection = sqlite3.connect('todo.db')
    cursor = connection.cursor()
    cursor.execute("SELECT id, task FROM todo WHERE status LIKE '1'" )
    result = cursor.fetchall()
    return str(result)
run(app, host='0.0.0.0', port=5000, debug=True)

In [ ]:
import sqlite3
from bottle import Bottle, run, template

app= Bottle()

@app.route('/todo')
@app.route('/')
def todo_list():
    connection = sqlite3.connect('todo.db')
    cursor = connection.cursor()
    cursor.execute("SELECT id, task FROM todo WHERE status LIKE '1'" )
    result = cursor.fetchall()
    output = template('make_table', rows=result)
    return output 
run(app, host='0.0.0.0', port=5000, debug=True)

In [ ]:
import sqlite3
from bottle import Bottle, run, template, request

app= Bottle()

@app.route('/todo')
@app.route('/')
def todo_list():
    connection = sqlite3.connect('todo.db')
    cursor = connection.cursor()
    cursor.execute("SELECT id, task FROM todo WHERE status LIKE '0'" )
    result = cursor.fetchall()
    output = template('make_table', rows=result)
    return output 

@app.route('/new', method='GET')
def new_item():
    return template('new_task.tpl')

@app.route('/new', method='POST')
def new_item_do():
    
    new_item = request.POST.task.strip()
    
    connection = sqlite3.connect('todo.db')
    cursor = connection.cursor()
    
    cursor.execute("INSERT INTO todo (task,status)VALUES (?,?)", (new_item, 0))
    new_id = cursor.lastrowid
    
    connection.commit()
    cursor.close()
    
    return f"""
    <meta HTTP-EQUIV="REFRESH" content="5; url={request.urlparts[0]+'://'+request.urlparts[1]}/">
    <p>The new task was insert into the database, the ID is {new_id}</p>"""
    

run(app, host='0.0.0.0', port=5000, debug=True)

In [ ]:
import sqlite3
from bottle import Bottle, run, template, request

app= Bottle()

@app.route('/todo')
@app.route('/')
def todo_list():
    connection = sqlite3.connect('todo.db')
    cursor = connection.cursor()
    cursor.execute("SELECT id, task FROM todo WHERE status LIKE '0'" )
    result = cursor.fetchall()
    output = template('make_table', rows=result)
    return output 

@app.route('/new', method='GET')
def new_item_do():
    
    if request.GET.save: 
    
        new_item = request.GET.task.strip()

        connection = sqlite3.connect('todo.db')
        cursor = connection.cursor()

        cursor.execute("INSERT INTO todo (task,status)VALUES (?,?)", (new_item, 0))
        new_id = cursor.lastrowid

        connection.commit()
        cursor.close()

        return f"""
        <meta HTTP-EQUIV="REFRESH" content="5; url={request.urlparts[0]+'://'+request.urlparts[1]}/">
        <p>The new task was insert into the database, the ID is {new_id}</p>"""
        
    else:
        return template('new_task.tpl')
    

run(app, host='0.0.0.0', port=5000, debug=True)

## Agregar una ruta para editar tareas

In [ ]:
import sqlite3
from bottle import Bottle, run, template, request

app= Bottle()

@app.route('/todo')
@app.route('/')
def todo_list():
    connection = sqlite3.connect('todo.db')
    cursor = connection.cursor()
    cursor.execute("SELECT id, task FROM todo WHERE status LIKE '0'" )
    result = cursor.fetchall()
    output = template('make_table', rows=result)
    return output 

@app.route('/new', method='GET')
def new_item_do():
    
    if request.GET.save: 
    
        new_item = request.GET.task.strip()

        connection = sqlite3.connect('todo.db')
        cursor = connection.cursor()

        cursor.execute("INSERT INTO todo (task,status)VALUES (?,?)", (new_item, 0))
        new_id = cursor.lastrowid

        connection.commit()
        cursor.close()

        return f"""
        <meta HTTP-EQUIV="REFRESH" content="5; url={request.urlparts[0]+'://'+request.urlparts[1]}/">
        <p>The new task was insert into the database, the ID is {new_id}</p>"""
        
    else:
        return template('new_task.tpl')
        
@app.route('/edit/<num:int>', method='GET')
def edit_item(num):
    
    if request.GET.save: 
    
        edit = request.GET.task.strip()
        status = request.GET.status.strip()
        
        if status == 'open':
            status = 0
        else:
            status = 1
            
        connection = sqlite3.connect('todo.db')
        cursor = connection.cursor()
        cursor.execute("UPDATE todo SET task = ?, status = ? WHERE id LIKE ?",(edit, status, num))        
        connection.commit()
            
        return f'<p>The item number {num} was successfully update</p>'
    else:
        connection=sqlite3.connect('todo.db')
        cursor=connection.cursor()
        cursor.execute("SELECT task FROM todo WHERE id LIKE ?",(str(num),))
        cur_data=cursor.fetchone()
        
        return template('edit_task',old=cur_data,num=num)
    


run(app, host='0.0.0.0', port=5000, debug=True)